In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%writefile space_invaders_dqn.yml
SpaceInvadersNoFrameskip-v4:
  env_wrapper:
    - stable_baselines3.common.atari_wrappers.AtariWrapper
  frame_stack: 4
  policy: 'CnnPolicy'
  n_timesteps: !!float 1e7
  buffer_size: 100000
  learning_rate: !!float 1e-4
  batch_size: 32
  learning_starts: 100000
  target_update_interval: 1000
  train_freq: 4
  gradient_steps: 1
  exploration_fraction: 0.1
  exploration_final_eps: 0.01
  # If True, you need to deactivate handle_timeout_termination
  # in the replay_buffer_kwargs
  optimize_memory_usage: False

Writing space_invaders_dqn.yml


In [ ]:
!python -m rl_zoo3.train --algo dqn --env SpaceInvadersNoFrameskip-v4 -f logs/ -c space_invaders_dqn.yml

========== SpaceInvadersNoFrameskip-v4 ==========
Seed: 3682618652
Loading hyperparameters from: space_invaders_dqn.yml
Default hyperparameters for environment (ones being tuned will be overridden):
OrderedDict([('batch_size', 32),
             ('buffer_size', 100000),
             ('env_wrapper',
              ['stable_baselines3.common.atari_wrappers.AtariWrapper']),
             ('exploration_final_eps', 0.01),
             ('exploration_fraction', 0.1),
             ('frame_stack', 4),
             ('gradient_steps', 1),
             ('learning_rate', 0.0001),
             ('learning_starts', 100),
             ('n_timesteps', 1000.0),
             ('optimize_memory_usage', False),
             ('policy', 'CnnPolicy'),
             ('target_update_interval', 1000),
             ('train_freq', 4)])
Using 1 environments
Creating test environment
A.L.E: Arcade Learning Environment (version 0.11.2+ecc1138)
[Powered by Stella]
Stacking 4 frames
Wrapping the env in a VecTransposeImage.
S

In [ ]:
!python -m rl_zoo3.enjoy --algo dqn --env SpaceInvadersNoFrameskip-v4 --no-render --n-timesteps 5000 --folder logs/

Loading latest experiment, id=1
Loading logs/dqn/SpaceInvadersNoFrameskip-v4_1/SpaceInvadersNoFrameskip-v4.zip
A.L.E: Arcade Learning Environment (version 0.11.2+ecc1138)
[Powered by Stella]
Stacking 4 frames
Atari Episode Score: 215.00
Atari Episode Length 2575
Atari Episode Score: 180.00
Atari Episode Length 2461
Atari Episode Score: 290.00
Atari Episode Length 2931
Atari Episode Score: 315.00
Atari Episode Length 3277
Atari Episode Score: 210.00
Atari Episode Length 2493
Atari Episode Score: 260.00
Atari Episode Length 2953
Atari Episode Score: 210.00
Atari Episode Length 2519


In [5]:
!python -m rl_zoo3.push_to_hub \
    --algo dqn \
    --env SpaceInvadersNoFrameskip-v4 \
    --repo-name dqn-SpaceInvadersNoFrameskip-v4 \
    --orga AntonDergunov \
    -f logs/

Loading latest experiment, id=1
Loading logs/dqn/SpaceInvadersNoFrameskip-v4_1/SpaceInvadersNoFrameskip-v4.zip
A.L.E: Arcade Learning Environment (version 0.11.2+ecc1138)
[Powered by Stella]
Stacking 4 frames
Wrapping the env in a VecTransposeImage.
Uploading to AntonDergunov/dqn-SpaceInvadersNoFrameskip-v4, make sure to have the rights
ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to some
minutes if video generation is activated. This is a work in progress: if you
encounter a bug, please open an issue.
Fetching 1 files:   0%|                                   | 0/1 [00:00<?, ?it/s]
.gitattributes: 1.52kB [00:00, 1.07MB/s]
Fetching 1 files: 100%|███████████████████████████| 1/1 [00:00<00:00,  1.70it/s]
Saving model to: hub/dqn-SpaceInvadersNoFrameskip-v4/dqn-SpaceInvadersNoFrameskip-v4
Saving video to /var/folders/f2/98g94n_x51l0zrxfnqp9s3wh0000gn/T/tmpb337g5a6/-step-0-to-step-1000.mp4
MoviePy -